# Using EcoFOCIpy to process raw field data

## DY1908 (Fall Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2019/CTDcasts/dy1904/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY1904' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY1904.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd013.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd014.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd017.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd018.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd021.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd024.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd026.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd027.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd028.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd029.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd036.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd037.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd038.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd039.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd040.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd041.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd042.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd043.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd044.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd045.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd046.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd047.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1904/rawconverted/ctd048.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sal00,sal11,sigma-t00,sigma-t11,fleco-afl,t090c,t190c,turbwetntu0,prdm,scan,datetime
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11
mean,6.550364,93.269636,285.280364,6.239355,88.841455,271.736000,32.335573,32.337764,25.441227,25.442882,16.779227,6.053109,6.053809,0.061073,174.346455,11883.545455,2019-04-20 01:53:59.090908928
min,6.546000,93.213000,285.089000,6.232200,88.733000,271.426000,32.334100,32.336300,25.440600,25.442200,15.683600,6.049000,6.049600,0.050900,174.070000,11664.000000,2019-04-20 01:53:50
25%,6.548000,93.242000,285.179000,6.236150,88.798500,271.594500,32.334400,32.336850,25.440750,25.442500,16.168000,6.049700,6.051300,0.056250,174.252000,11794.500000,2019-04-20 01:53:55.500000
50%,6.550100,93.257000,285.268000,6.239000,88.839000,271.722000,32.336300,32.338300,25.441500,25.443100,16.867700,6.054900,6.054900,0.060300,174.285000,11881.000000,2019-04-20 01:53:59
75%,6.552800,93.303500,285.387000,6.242400,88.890000,271.868000,32.336400,32.338450,25.441550,25.443200,17.163750,6.055300,6.055850,0.064600,174.509000,11973.500000,2019-04-20 01:54:03
max,6.554800,93.325000,285.472000,6.247600,88.966000,272.096000,32.336500,32.338600,25.441700,25.443200,18.094400,6.056300,6.056700,0.074100,174.640000,12107.000000,2019-04-20 01:54:08
std,0.002950,0.039434,0.128572,0.005124,0.073864,0.223438,0.001047,0.000965,0.000452,0.000421,0.748396,0.003009,0.002799,0.007579,0.196731,132.321853,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY1904',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2019, 4, 19),
 'EndDate': datetime.date(2019, 5, 1),
 'Project': 'EcoFOCI',
 'ChiefScientist': 'Peter Proctor',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Spring Mooring Survey',
 'CruiseYear': 2019,
 'ctdlogs_pdf_name': 'DY1904_CastLogs.pdf'}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 24017,
 'Vessel': 'R/V Oscar Dyson',
 'CruiseID': 'DY1904',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY1904',
 'Project': 'EcoFOCI Spring Moorings',
 'StationNo_altname': 'S1H1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 57,
 'LatitudeMin': 43.35,
 'LongitudeDeg': 152,
 'LongitudeMin': 17.71,
 'GMTDay': 20,
 'GMTMonth': 'Apr',
 'GMTYear': 2019,
 'GMTTime': 6840,
 'DryBulb': 4.6,
 'RelativeHumidity': 57,
 'WetBulb': -99.9,
 'Pressure': 1006,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': 38,
 'WindSpd': 9.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 183,
 'StationNameID': '19CB1A',
 'MaxDepth': 172,
 'InstrumentSerialNos': 'Press SN =0772, Pri Temp SN = 4379, Sec Temp SN =2376, Pri Cond SN = 2985, Sec Cond Sn =3127, PAR Sn =70547, Fluor Sn = FLNTU-2057, pri O2 Sn = 1961, sec O2 Sn = 3200 ',
 'Notes': 'Post Deployment Chiniak Mooring.  During titration of O2 samples, 305 failed.\r\nTest all bottles, 7 hung on fir

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [9]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [10]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,Pressure [dbar],scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
1.0,6.5501,93.257,285.268,6.2383,88.818,271.689,32.3342,32.3364,25.4406,25.4423,16.6591,6.0496,6.0497,0.0509,174.604,11664.0,2019-04-20 01:53:50,ctd001


In [11]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'salinity_ch1', 'salinity_ch2',
       'sigma_t_ch1', 'sigma_t_ch2', 'chlor_fluorescence', 'temperature_ch1',
       'temperature_ch2', 'turbidity', 'Pressure [dbar]', 'scan', 'datetime',
       'cast'],
      dtype='object')

## Add institutional meta-information


In [12]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [13]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [14]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (bottle: 11)
Coordinates:
  * bottle              (bottle) float64 88B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) float64 88B 6.55 6.55 6.553 ... 6.546 6.549
    oxy_percentsat_ch1  (bottle) float64 88B 93.26 93.25 93.3 ... 93.21 93.25
    oxy_conc_ch1        (bottle) float64 88B 285.3 285.3 285.4 ... 285.1 285.2
    oxy_concM_ch2       (bottle) float64 88B 6.238 6.246 6.24 ... 6.241 6.235
    oxy_percentsat_ch2  (bottle) float64 88B 88.82 88.93 88.84 ... 88.88 88.78
    oxy_conc_ch2        (bottle) float64 88B 271.7 272.0 271.8 ... 271.8 271.5
    ...                  ...
    temperature_ch2     (bottle) float64 88B 6.05 6.05 6.05 ... 6.055 6.055
    turbidity           (bottle) float64 88B 0.0509 0.0741 ... 0.0677 0.0615
    Pressure [dbar]     (bottle) float64 88B 174.6 174.2 174.1 ... 174.3 174.3
    scan                (bottle) float64 88B 1.166e+04 1.174e+04 ... 1.211e+04
    datetime            (bottle) datetime64[ns] 88B 2019-04-20T01:53:50 ... 2...
    cast                (bottle) object 88B 'ctd001' 'ctd001' ... 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [15]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [16]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [17]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 11)
Coordinates:
  * bottle              (bottle) float64 88B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 88B 6.55 ...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 88B 93.26...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 88B 285.3...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 88B 6.238...
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 88B 88.82...
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 88B 271.7...
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 88B 6.05 ...
    turbidity           (latitude, longitude, time, bottle) float64 88B 0.050...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 88B 174.6...
    scan                (latitude, longitude, time, bottle) float64 88B 1.166...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 88B ...
    cast                (latitude, longitude, time, bottle) object 88B 'ctd00...

In [18]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 11)
Coordinates:
  * bottle              (bottle) float64 88B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 88B 6.55 ...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 88B 93.26...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 88B 285.3...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 88B 6.238...
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 88B 88.82...
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 88B 271.7...
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 88B 6.05 ...
    turbidity           (latitude, longitude, time, bottle) float64 88B 0.050...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 88B 174.6...
    scan                (latitude, longitude, time, bottle) float64 88B 1.166...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 88B ...
    cast                (latitude, longitude, time, bottle) object 88B 'ctd00...
Attributes: (12/44)
    CruiseID:                      DY1904
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-12-03T08:04:58Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [19]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history